<a href="https://colab.research.google.com/github/nderakhshan/EDOS/blob/main/DBert01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')
from transformers import AutoTokenizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 91.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.8 MB/s eta 0:00:00


In [ ]:
cleandata = pd.read_csv('https://raw.githubusercontent.com/nderakhshan/EDOS/main/cleandata%20-%20cleandata.csv')

In [ ]:
print(cleandata)
dfTrain=cleandata[cleandata['split']=="train"]
dfVal=cleandata[cleandata['split']=="dev"]
dfTest=cleandata[cleandata['split']=="test"]

                      rewire_id  \
0       sexism2022_english-9609   
1      sexism2022_english-16993   
2      sexism2022_english-13149   
3      sexism2022_english-13021   
4        sexism2022_english-966   
...                         ...   
19995   sexism2022_english-5228   
19996  sexism2022_english-10140   
19997   sexism2022_english-9726   
19998  sexism2022_english-13365   
19999   sexism2022_english-3523   

                                                    text  label_sexist  \
0      in nigeria if you rape a woman the men rape yo...             0   
1                                   then she is a keeper             0   
2      this is like the metallica video where the poo...             0   
3                                                  woman             0   
4                         i bet she wished she had a gun             0   
...                                                  ...           ...   
19995       girls really get fucked almost every weekend     

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
batch0 = cleandata[:2000]
labels = cleandata['label_sexist']
labels=labels[:2000]
batch0['label_sexist'].value_counts()

0    2098
1     902
Name: label_sexist, dtype: int64

In [ ]:
tokenized = batch0['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
np.array(padded).shape
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape
input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression()

In [ ]:
pred=lr_clf.predict (test_features)

In [ ]:
f1_score(test_labels, pred)

0.4850136239782017